In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("CalculateGeoFeature") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/06/01 10:12:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/01 10:12:26 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/01 10:12:26 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/01 10:12:26 WARN util.Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/06/01 10:12:26 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
from datetime import datetime

date_string = datetime.today().strftime('%Y-%m-%d')

In [3]:
user_geo_feature = spark.sql(f'''
select
    uid as user_id,
    province,
    city,
    district,
    '{date_string}' as date
from
    user_ods
where
    date = '{date_string}'
''')
user_geo_feature.write.mode("overwrite").partitionBy("date").saveAsTable("user_geo_feature")

23/06/01 10:12:54 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [4]:
item_geo_feature = spark.sql(f'''
select
    id as item_id,
    province,
    city,
    district,
    '{date_string}' as date
from
    item_ods
where
    date = '{date_string}'
''')
item_geo_feature.write.mode("overwrite").partitionBy("date").saveAsTable("item_geo_feature")

In [5]:
spark.stop()